In [1]:
from tensorflow.keras import layers
from tensorflow.keras import models
import tensorflow as tf
from tensorflow import keras
import json
with open('constants.json') as json_file:
    data = json.load(json_file)
    label_names = data["label_names"]
from os import path 
from pydub import AudioSegment 
import numpy as np
from random import randint

def get_spectrogram(waveform):
  # Convert the waveform to a spectrogram via a STFT.
  spectrogram = tf.signal.stft(
      waveform, frame_length=255, frame_step=128)
  # Obtain the magnitude of the STFT.
  spectrogram = tf.abs(spectrogram)
  # Add a `channels` dimension, so that the spectrogram can be used
  # as image-like input data with convolution layers (which expect
  # shape (`batch_size`, `height`, `width`, `channels`).
  spectrogram = spectrogram[..., tf.newaxis]
  return  spectrogram

def import_model():

    model = tf.keras.models.load_model('blankmodel.keras')
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'],
    )

    checkpoint_filepath = 'model.keras'
    model.load_weights(checkpoint_filepath)
    return model

def get_prediction():
    model = import_model()

    x = tf.io.read_file(str('voice.wav'))
    x, _ = tf.audio.decode_wav(x, desired_channels=1, desired_samples=80000,)
    x = tf.squeeze(x, axis=-1)
    x = get_spectrogram(x)
    x = x[tf.newaxis,...]
    probability_model = tf.keras.Sequential([model, 
                                            tf.keras.layers.Softmax()])

    prediction = probability_model.predict(x)
    return prediction

def to_json(prediction):
    y = dict(zip(label_names, prediction.tolist()[0]))
    x = randint(0, 1000)

    sorted_dict = dict(sorted(y.items(), key=lambda item: item[1], reverse=True))

    filtered_data = {key: value for key, value in sorted_dict.items() if value > 0.08}
    if x == 100:
        filtered_data.update({"coconut":"0.01"})

    y = json.dumps(filtered_data)
    return y


if __name__ == "__main__":
    input_file = "voice.mp3"
    output_file = "voice.wav"
        
    try:
        open(input_file, "r")   
        sound = AudioSegment.from_mp3(input_file) 
        sound.export(output_file, format="wav") 
        prediction = get_prediction()
        y = to_json(prediction)


    except FileNotFoundError:
        y = json.dumps({"error":"File not found"})

    
    with open("output.json", "w") as outfile: 
        outfile.write(y)



c:\Users\hanna\miniconda3\envs\custom_py\Lib\site-packages\keras\src\saving\saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
